In [ ]:
import gensim
import os
import collections
import smart_open
import random

import pandas as pd
import numpy as np


from pprint import pprint
import string

# TODO
- Remove punctuations
- Filtering based on word count

In [ ]:
def get_train_data():
    """
    SQL here 
    """
    data = pd.read_csv('~/Desktop/grocery_products.csv')
    return data

In [ ]:
data = get_train_data()

In [ ]:
print(data.shape)

In [307]:
selected_data = data[['source_product_id', 'name', 'departments_name', 'categories_name', 'description_en']]
selected_data = selected_data.head(1000)

# # replace nan values with ''
selected_data = selected_data.replace(np.nan, '', regex=True)
selected_data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1000 entries, 0 to 999
Data columns (total 5 columns):
source_product_id    1000 non-null int64
name                 1000 non-null object
departments_name     1000 non-null object
categories_name      1000 non-null object
description_en       1000 non-null object
dtypes: int64(1), object(4)
memory usage: 39.1+ KB


Creating document string

In [308]:
selected_data['document'] = selected_data['name'] + ' ' + selected_data['departments_name'] + ' ' + selected_data['categories_name'] + ' ' + selected_data['description_en']

In [343]:
selected_data[selected_data['source_product_id'] == 4979224]

,source_product_id,name,departments_name,categories_name,description_en,document
763,4979224,Tea Infusions - Ginger Tea,Drinks,Tea,Super uses sophisticated technology in their t...,Tea Infusions - Ginger Tea Drinks Tea Super us...


In [ ]:
docs = selected_data['document'].tolist()
product_ids = selected_data['source_product_id'].tolist()

# Mapping from document index 
doc_index_to_id = {}
for index, product_id in enumerate(product_ids):
    doc_index_to_id[index] = product_id

In [342]:
doc_index_to_id[763]

4979224

In [332]:
pp.pprint(docs[2])
# print(type(docs[0]))

[ 'ice',
  'lemon',
  'tea',
  'drinks',
  'tea',
  'refreshing',
  'combination',
  'fruity',
  'flavours',
  'real',
  'tea',
  'leaves',
  'sweet',
  'cooling',
  'tea',
  'taste',
  'with',
  'stimulating',
  'lemon',
  'flavour',
  'finish']


In [312]:
pprint(docs)

['A08. Golden Nasi Super Combo Department - (Mchefz Cuisine Menu '
 '(mchefz-cuisine)) Rice Served with fried chicken thigh and braised beef and '
 'cuttlefish, add ons available',
 'Honey Stars Breakfast Cereal Food Cupboard Cereals & Cereal Bars This '
 "product is getting close to its expiry date—don't let this get away from "
 "you! It's still safe for consumption, now enjoy it at a lower "
 'price.<br/><br/>Expiry Date: 2018-07-30<br/><br/>',
 'Ice Lemon Tea Drinks Tea Refreshing combination of fruity flavours and real '
 'tea leaves. Sweet cooling tea taste with a stimulating lemon flavour finish',
 'Anticavity Maximum Cavity Protection Toothpaste Health & Beauty Oral Care '
 "This product is getting close to its expiry date—don't let this get away "
 "from you! It's still safe for use, now enjoy it at a lower price.<br/> "
 '<br/>Expiry Date : 2018-10-13',
 'Daily Moisture Therapy Dermo-Cleanser Twin Pack Health & Beauty Body Care '
 'Daily Moisture Therapy Dermo-Cleanser is a h

 'Ingredients: Japanese citrus "kawachi-bankan"',
 'Black Chia Seeds Members Specials Food Cupboard Prana Chia whole black:Chia '
 'seeds (Salvia Hispanica) originate from the central valleys of Mexico and '
 'has been known and used for over 5,000 years. It was used as a survival food '
 'by the Aztecs - it was said that a tablespoon of chia could keep a warrior '
 'strong for 24 hours. It was also used as an offering to the gods.',
 'Muruku Shevel Snacks & Sweets Crisps & Popcorn From humble beginnings in '
 'both West and South India, this savoury nut snack has grown in popularity. '
 'Using only the finest and freshest ingredients, this festive treat is now '
 'available all year round. It’s crunchy, tasty and perfect for sharing. Truly '
 'authentic and truly tasty! Camel believe in sourcing only high quality raw '
 'materials from around the world. Producing anytime, anywhere snacks that are '
 'as natural as they are nutritious.',
 'Original Baked Cheesecake Members Specials Fro

 'Peas Special Grade is made only from the freshest and premium green peas. An '
 'excellent source of minerals and antioxidant vitamins, and dietary fibre. '
 'Cooked and ready to eat. This quick and versatile side dish is perfect with '
 'fried pork chops, chicken, and main dishes. Also perfect for filling and '
 'nutritious soup recipes.',
 "Enriched Butter Sugar Wholemeal Cream Roll Bakery Bread Gardenia's enriched "
 'butter sugar wholemeal cream rolls are soft and fresh - the perfect choice '
 'for sweet snack or dessert. Perfectly portioned, so you can enjoy a fresh '
 'cream roll whenever you want. Gardenia has been perfecting their fresh baked '
 "recipes in Singapore since 1978. They've since grown to become one of the "
 'top bread producers in Southeast Asia, with a range of innovative and tasty '
 'baked goods.',
 '(DELETED) Australia Pork Minced Fresh Meat Pork Country of Origin: Australia',
 'Swiss Premium Semi-Skimmed Milk Drinks Milk Products & Soy Drinks This '
 "prod

In [313]:
def clean_documents(docs):
    """
    docs:
        list of string documents
    
    """
    # Remove punctuations
    translator = str.maketrans('', '', string.punctuation)
    docs = [d.translate(translator) for d in docs if isinstance(d, str)]
    
    # remove common words and tokenize
    stoplist = set('for a of the and to in'.split())
    docs = [[word for word in document.lower().split() if word not in stoplist]
             for document in docs]

    # remove words that appear only once
    from collections import defaultdict
    frequency = defaultdict(int)
    for text in docs:
        for token in text:
            # TODO : Right now increasing by 2 to not skip any word
            frequency[token] += 2

    docs = [[token for token in doc if frequency[token] > 1] for doc in docs]

    pprint(docs)
    return docs

In [314]:
docs = clean_documents(docs)

[['a08',
  'golden',
  'nasi',
  'super',
  'combo',
  'department',
  'mchefz',
  'cuisine',
  'menu',
  'mchefzcuisine',
  'rice',
  'served',
  'with',
  'fried',
  'chicken',
  'thigh',
  'braised',
  'beef',
  'cuttlefish',
  'add',
  'ons',
  'available'],
 ['honey',
  'stars',
  'breakfast',
  'cereal',
  'food',
  'cupboard',
  'cereals',
  'cereal',
  'bars',
  'this',
  'product',
  'is',
  'getting',
  'close',
  'its',
  'expiry',
  'date—dont',
  'let',
  'this',
  'get',
  'away',
  'from',
  'you',
  'its',
  'still',
  'safe',
  'consumption',
  'now',
  'enjoy',
  'it',
  'at',
  'lower',
  'pricebrbrexpiry',
  'date',
  '20180730brbr'],
 ['ice',
  'lemon',
  'tea',
  'drinks',
  'tea',
  'refreshing',
  'combination',
  'fruity',
  'flavours',
  'real',
  'tea',
  'leaves',
  'sweet',
  'cooling',
  'tea',
  'taste',
  'with',
  'stimulating',
  'lemon',
  'flavour',
  'finish'],
 ['anticavity',
  'maximum',
  'cavity',
  'protection',
  'toothpaste',
  'health',
  'b

  'products',
  'we',
  'are',
  'here',
  'protect',
  'enhance',
  'it',
  'we',
  'support',
  'fastronomic',
  'excellence',
  'our',
  'land',
  'with',
  'enthusiasm',
  'so',
  'that',
  'original',
  'balsamic',
  'venegar',
  'modena',
  'remains',
  'jewel',
  'local',
  'food'],
 ['la',
  'bergere',
  'de',
  'thones',
  'dairy',
  'eggs',
  'chilled',
  'food',
  'cheese'],
 ['basmati',
  'rice',
  'extra',
  'long',
  'members',
  'specials',
  'food',
  'cupboard',
  'basmati',
  'is',
  'known',
  'as',
  'fragrant',
  'one',
  'among',
  'other',
  'varieties',
  'it',
  'has',
  'distinctive',
  'appealing',
  'flavour',
  'which',
  'makes',
  'it',
  'favourite',
  'choice',
  'chefs',
  'longer',
  'grain',
  'when',
  'cooked',
  'becomes',
  'much',
  'fluffier',
  'less',
  'sticky',
  'gets',
  'easily',
  'seperated'],
 ['602',
  'beef',
  'bolognise',
  'spaghetti',
  'department',
  'chicken',
  'king',
  'menu',
  '2',
  'chickenkingsg',
  'spaghetti'],
 ['f

  'just',
  'spoil',
  'your',
  'friends',
  'with',
  'hot',
  'chicken',
  'snack',
  'afternoon'],
 ['complete',
  '1',
  'multi',
  'vitamin',
  'health',
  'beauty',
  'supplements',
  'nutrition'],
 ['herbs',
  'black',
  'pepper',
  'coarse',
  'food',
  'cupboard',
  'cooking',
  'ingredients',
  'established',
  'canada',
  '1889',
  'by',
  'william',
  'schwartz',
  'son',
  'german',
  'immigrant',
  'schwartz',
  'brand',
  'is',
  'global',
  'leader',
  'its',
  'field',
  'today',
  'it',
  'is',
  'one',
  'world’s',
  'largest',
  'producers',
  'herbs',
  'spices',
  'flavourings',
  'seasonings',
  'company',
  'believes',
  'supplying',
  'small',
  'quantity',
  'top',
  'quality',
  'spices',
  'rather',
  'than',
  'poor',
  'quality',
  'bulk',
  'products',
  'black',
  'pepper’s',
  'bold',
  'flavour',
  'adds',
  'powerful',
  'kick',
  'meats',
  'oven',
  'roasted',
  'vegetables',
  'add',
  'dash',
  'black',
  'pepper',
  'splash',
  'balsamic',
  'vi

  'you',
  'your',
  'partner',
  'closer',
  'together'],
 ['diapers', 'm', 'diapers', 'diapers', 'reusable', 'nappies'],
 ['hot',
  'cup',
  'noodle',
  'chicken',
  'flavour',
  'food',
  'cupboard',
  'rice',
  'noodles',
  'pasta',
  'rolls',
  'this',
  'product',
  'is',
  'getting',
  'close',
  'its',
  'expiry',
  'date—dont',
  'let',
  'this',
  'get',
  'away',
  'from',
  'you',
  'its',
  'still',
  'safe',
  'consumption',
  'now',
  'enjoy',
  'it',
  'at',
  'lower',
  'pricebr',
  'br',
  'enjoy',
  'with',
  'traditional',
  'taste',
  'homemade',
  'malaysian',
  'flavours',
  'with',
  'maggi',
  'hot',
  'cup',
  'noodle',
  'chicken',
  'flavour',
  'this',
  'chicken',
  'noodle',
  'is',
  'made',
  'with',
  'real',
  'chicken',
  'essence',
  'aromatic',
  'herbs',
  'it’s',
  'an',
  'excellent',
  'timesaver',
  'anyone',
  'convenient',
  'easy',
  'prepare',
  'garnish',
  'with',
  'boiled',
  'egg',
  'or',
  'green',
  'onions',
  'enhance',
  'flavou

  'it',
  'smells',
  'rich',
  'sweet',
  'when',
  'applied',
  'this',
  'is'],
 ['natural',
  'extract',
  'moroccan',
  'almond',
  'food',
  'cupboard',
  'baking',
  'ingredients',
  'sunflower',
  'oil',
  'extract',
  'almond'],
 ['malacca',
  'nyonya',
  'sweet',
  'spicy',
  'sauce',
  'food',
  'cupboard',
  'sauces',
  'dressings',
  'always',
  'keep',
  'pack',
  'hengs',
  'malacca',
  'nyonya',
  'sweet',
  'spicy',
  'sauce',
  'easytocook',
  'sweet',
  'spicy',
  'seafood',
  'this',
  'hot',
  'sauce',
  'is',
  'made',
  'from',
  'chillies',
  'spices',
  'herbs',
  'aromatics',
  'it',
  'has',
  'no',
  'added',
  'msg',
  'or',
  'added',
  'colouring',
  'perfect',
  'shrimp',
  'satay',
  'or',
  'grilled',
  'fish',
  'vegetables',
  'convenient',
  'easy',
  'use',
  'certified',
  'halal'],
 ['aqua',
  'mist',
  'floral',
  'bouquet',
  'household',
  'cleaning',
  'air',
  'fresheners',
  'this',
  'aqua',
  'mist',
  'spray',
  'is',
  'multiuse',
  'sc

...

  'syrup',
  'salt',
  'vegetable',
  'fat',
  'miso',
  'garlic',
  'powder',
  'spice',
  'ginger',
  'powder',
  'onion',
  'powder',
  'bonito',
  'flake',
  'seasoning',
  'kansui',
  'caramel',
  'pigment',
  'thickener'],
 ['mozzarella',
  'fiorello',
  'members',
  'specials',
  'dairy',
  'eggs',
  'chilled',
  'food',
  'our',
  'cheeses',
  'feature',
  'finest',
  'fresh',
  'natural',
  'ingredients',
  'are',
  'made',
  'according',
  'timehonoured',
  'italian',
  'recipes'],
 ['japan',
  'apple',
  'juice',
  'drinks',
  'juices',
  'juice',
  'drinks',
  'winner',
  'superior',
  'tasta',
  'award',
  'seikans',
  '100',
  'natural',
  'apple',
  'juice',
  'is',
  'produced',
  'with',
  'care',
  'japan',
  'pressed',
  'from',
  'best',
  'fresh',
  'aomori',
  'apples',
  'their',
  'skins',
  'this',
  'juice',
  'is',
  'sweet',
  'mellow',
  'with',
  'no',
  'additives',
  'preservatives',
  'or',
  'sweeteners',
  'this',
  'juice',
  'is',
  'an',
  'easy',


  'bacon',
  'that',
  'has',
  'been',
  'cured',
  'with',
  'water',
  'salt',
  'sugar',
  'sodium',
  'erythorbate',
  'sodium',
  'nitrite',
  'it',
  'may',
  'also',
  'contain',
  'smoke',
  'flavoring',
  'dextrose',
  'brown',
  'sugar',
  'sodium',
  'phosphate',
  'potassium',
  'chloride',
  'sodium',
  'diacetate',
  'flavoring',
  'honey'],
 ['scotch',
  'brite',
  'utility',
  'brush',
  'home',
  'housekeeping',
  'this',
  'scotchbrite',
  'utility',
  'brush',
  'is',
  'perfect',
  'cleaning',
  'sweeping',
  'tough',
  'hardtoreach',
  'areas',
  'with',
  'an',
  'ergonomic',
  'handle',
  'better',
  'gripping',
  'action',
  'this',
  'brush',
  'will',
  'help',
  'you',
  'sweep',
  'scrup',
  'floors',
  'walls',
  'with',
  'ease',
  'its',
  'bristles',
  'are',
  'tough',
  'enough',
  'give',
  'deep',
  'clean',
  'without',
  'leaving',
  'scratches'],
 ['10',
  'chicken',
  'franks',
  'original',
  'frozen',
  'food',
  'convenience',
  'food',
  'st

  'with',
  'eastern',
  'mediterranean',
  'lemon',
  'balm',
  'added',
  'fresh',
  'fragrance',
  'flavour',
  'available',
  'handy',
  'resealable',
  'bag'],
 ['black',
  'or',
  'white',
  'coffee',
  '黑白咖啡',
  'department',
  'shao',
  '烧',
  'menu',
  'shao',
  'beverages',
  '饮料'],
 ['twirl',
  'chocolate',
  'bar',
  '5',
  'pack',
  'snacks',
  'sweets',
  'chocolate',
  'ribbons',
  'creamy',
  'milk',
  'chocolate',
  'are',
  'draped',
  'more',
  'milky',
  'chocolate',
  'an',
  'unbeatable',
  'meltinthemouth',
  'experience',
  'flaky',
  'chocolate',
  'ribbons',
  'offer',
  'unique',
  'texture',
  'ultimate',
  'sweet',
  'escape',
  'crisp',
  'with',
  'delicate',
  'crunch',
  'cadbury',
  'twirl',
  'chocolate',
  'ma'],
 ['enfamil', 's2', '900g', 'baby', 'milk', 'powder', 'stage', '2'],
 ['mocha',
  'gold',
  'mild',
  'coffee',
  'mix',
  'drinks',
  'coffee',
  'maxim',
  'mocha',
  'gold',
  'mild',
  'coffee',
  'mix',
  'is',
  'korean',
  'blend',
  '

 ['fairtrade',
  'cane',
  'sugar',
  'demerara',
  'cubes',
  'food',
  'cupboard',
  'sugar',
  'substitutes',
  'syrup',
  'fairtrade',
  'cane',
  'sugar'],
 ['chunkie',
  'extremely',
  'chocolatey',
  'cookies',
  'snacks',
  'sweets',
  'cookies',
  'biscuits',
  'friends',
  'over',
  'your',
  'nearest',
  'dearest',
  'deserve',
  'something',
  'special',
  'that',
  'calls',
  'fox’s',
  'cookies',
  'with',
  'their',
  'rugged',
  'chunks',
  'chocolate',
  'our',
  'chunkie',
  'cookies',
  'will',
  'make',
  'you',
  'very',
  'popular',
  'whether',
  'you’re',
  'meeting',
  'up',
  'gossip',
  'or',
  'having',
  'cosy',
  'night'],
 ['vienna', 'almonds', 'new', 'arrivals', 'snacks', 'sweets'],
 ['glass',
  'cleaner',
  'lavender',
  'household',
  'cleaning',
  'cleaning',
  'supplies',
  'mrs',
  'meyers',
  'clean',
  'day',
  'glass',
  'cleaner',
  'cleans',
  'shines',
  'mirrors',
  'other',
  'glass',
  'surfaces',
  'inside',
  'outside',
  'home',
  'this'

  'soapy',
  'floral',
  'fragrance'],
 ['pocky',
  'crush',
  'almond',
  'snacks',
  'sweets',
  'cookies',
  'biscuits',
  'ingredients',
  'wheat',
  'flour',
  'sugar',
  'almond',
  'whole',
  'milk',
  'powder',
  'vegetable',
  'oil',
  'cacaomass',
  'coco',
  'butter',
  'shortening',
  'salt',
  'butter',
  'yeast',
  'emulsifiers',
  'flavoring',
  'annatto',
  'pigment',
  'baking',
  'powder',
  'starch',
  'mono',
  'calcium',
  'phosphate',
  'sodium',
  'bicarbonate'],
 ['bottled',
  'drink',
  'honey',
  'green',
  'tea',
  'drinks',
  'tea',
  'pokka',
  'honey',
  'green',
  'tea',
  'is',
  '100',
  'real',
  'brewed',
  'blends',
  'sweet',
  'luscious',
  'honey',
  'with',
  'green',
  'tea',
  'endearing',
  'refreshment',
  'that',
  'lingers',
  'on',
  'your',
  'lips',
  'sweeten',
  'your',
  'life',
  'with',
  'honey',
  'goodness',
  'today',
  'it',
  'contains',
  'antioxidants',
  'no',
  'added',
  'preservatives',
  'nor',
  'colouring'],
 ['winter

In [315]:
dictionary = corpora.Dictionary(docs)
dictionary.save(os.path.join('output_data', 'products_corpus.dict'))

In [316]:
print(dictionary)

Dictionary(5739 unique tokens: ['a08', 'add', 'available', 'beef', 'braised']...)


In [317]:
print(dictionary.token2id)

{'a08': 0, 'add': 1, 'available': 2, 'beef': 3, 'braised': 4, 'chicken': 5, 'combo': 6, 'cuisine': 7, 'cuttlefish': 8, 'department': 9, 'fried': 10, 'golden': 11, 'mchefz': 12, 'mchefzcuisine': 13, 'menu': 14, 'nasi': 15, 'ons': 16, 'rice': 17, 'served': 18, 'super': 19, 'thigh': 20, 'with': 21, '20180730brbr': 22, 'at': 23, 'away': 24, 'bars': 25, 'breakfast': 26, 'cereal': 27, 'cereals': 28, 'close': 29, 'consumption': 30, 'cupboard': 31, 'date': 32, 'date—dont': 33, 'enjoy': 34, 'expiry': 35, 'food': 36, 'from': 37, 'get': 38, 'getting': 39, 'honey': 40, 'is': 41, 'it': 42, 'its': 43, 'let': 44, 'lower': 45, 'now': 46, 'pricebrbrexpiry': 47, 'product': 48, 'safe': 49, 'stars': 50, 'still': 51, 'this': 52, 'you': 53, 'combination': 54, 'cooling': 55, 'drinks': 56, 'finish': 57, 'flavour': 58, 'flavours': 59, 'fruity': 60, 'ice': 61, 'leaves': 62, 'lemon': 63, 'real': 64, 'refreshing': 65, 'stimulating': 66, 'sweet': 67, 'taste': 68, 'tea': 69, '20181013': 70, 'anticavity': 71, 'beaut

In [318]:
corpus = [dictionary.doc2bow(doc) for doc in docs]
corpora.MmCorpus.serialize(os.path.join('output_data', 'products_corpus.mm'), corpus)

In [319]:
! ls 'output_data'

deerwester.dict          products_corpus.mm
products_corpus.dict     products_corpus.mm.index


In [320]:
DATA_FOLDER = 'output_data'
dictionary = corpora.Dictionary.load(os.path.join(DATA_FOLDER, 'products_corpus.dict'))
corpus = corpora.MmCorpus(os.path.join(DATA_FOLDER, 'products_corpus.mm'))

In [321]:
print(corpus)

MmCorpus(1000 documents, 5739 features, 29934 non-zero entries)


In [322]:
lsi = models.LsiModel(corpus, id2word=dictionary, num_topics=20)
index = similarities.MatrixSimilarity(lsi[corpus])

/Users/vipul/anaconda3/lib/python3.7/site-packages/gensim/matutils.py:737: FutureWarning: Conversion of the second argument of issubdtype from `int` to `np.signedinteger` is deprecated. In future, it will be treated as `np.int64 == np.dtype(int).type`.
  if np.issubdtype(vec.dtype, np.int):


### Testing a sample document for similarity

In [323]:
sample_doc = 'heathy tea'

In [324]:
vec_bow = dictionary.doc2bow(sample_doc.lower().split())
vec_lsi = lsi[vec_bow]
print(vec_lsi)

[(0, 0.06485279418759622), (1, -0.024894430699521655), (2, -0.14104601107731113), (3, 0.1666035140153082), (4, 0.09456997697037875), (5, 0.15430919039701843), (6, 0.2647939855589868), (7, -0.26164625927739615), (8, 0.34325260594087226), (9, 0.16152970054405588), (10, -0.13054117263099388), (11, 0.51730187666932), (12, 0.05738882853345042), (13, 0.004665419223761937), (14, -0.24970888615608466), (15, 0.09611118486309854), (16, -0.015237941974697657), (17, 0.03697868222424273), (18, -0.06076422406160661), (19, -0.08552871956591926)]


In [325]:
similarity_scores = index[vec_lsi]
similarity_scores = list(enumerate(similarity_scores))

In [326]:
# Sort in descending order of similarity
sorted_similarity_scores = sorted(similarity_scores, key=lambda tup: tup[1], reverse=True)

In [350]:
print(sorted_similarity_scores[:10])

[(476, 0.94976234), (763, 0.9370842), (2, 0.9226046), (240, 0.9226046), (962, 0.9093931), (597, 0.904487), (328, 0.90292656), (841, 0.8961064), (396, 0.8856966), (727, 0.8843177)]


In [345]:
type(sorted_similarity_scores[0])

tuple

In [351]:
for doc_index, score in sorted_similarity_scores[:10]:
    product_id = doc_index_to_id[doc_index]
    product_info = str(selected_data[selected_data['source_product_id'] == product_id]['name'])
    print(f'{product_info} : {score}')

476    Tea - Rosehip & Hibiscus
Name: name, dtype: object : 0.9497623443603516
763    Tea Infusions - Ginger Tea
Name: name, dtype: object : 0.9370841979980469
2    Ice Lemon Tea
Name: name, dtype: object : 0.9226046204566956
240    Ice Lemon Tea
Name: name, dtype: object : 0.9226046204566956
962    Mango Tea
Name: name, dtype: object : 0.9093930721282959
597    Packet Drink - Jasmine Green Tea (Less Sugar)
Name: name, dtype: object : 0.9044870138168335
328    Tea - Four Red Fruit
Name: name, dtype: object : 0.9029265642166138
841    Instant Drink - Lemon tea
Name: name, dtype: object : 0.8961064219474792
396    Green Tea Zero with Sweeteners
Name: name, dtype: object : 0.8856965899467468
727    Jasmine Green Tea Drink
Name: name, dtype: object : 0.8843176960945129
